In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import random
import cv2
import pydicom

import warnings
warnings.filterwarnings("ignore")

In [ ]:
import pandas as pd
import seaborn as sns
from tqdm import tqdm


In [ ]:
import matplotlib.pyplot as plt
# from utilities_x_ray import read_xray,showXray


In [ ]:
import numpy as np


In [ ]:
train = pd.read_csv('../input/vinbigdata-chest-xray-abnormalities-detection/train.csv')
ss = pd.read_csv('../input/vinbigdata-chest-xray-abnormalities-detection/sample_submission.csv')

In [ ]:
train.head()

In [ ]:
train.shape

In [ ]:
train.image_id.describe()

In [ ]:
train.class_name.value_counts()

In [ ]:
train.rad_id.value_counts()

In [ ]:
reliable_annotators2 = ['R8', 'R9', 'R10']
reliable_annotators = ['R9']


In [ ]:
train_reliable = train[train.rad_id.isin(reliable_annotators)]
train_reliable2 = train[train.rad_id.isin(reliable_annotators2)]


In [ ]:
train_reliable.shape

In [ ]:
train_reliable2.shape

In [ ]:
train_reliable.class_name.value_counts()

In [ ]:
train_reliable2.class_name.value_counts()

In [ ]:
# train = train_reliable
train = train_reliable2


In [ ]:
print("Number of rows in train dataframe: {}".format(train.shape[0]))
print("Number of Unique images in train set: {}".format(train.image_id.nunique()))
print("Number of Classes: {}\n".format(train.class_name.nunique()))
print("Class Names: {}".format(list(train.class_name.unique())))

In [ ]:
class_names = sorted(train.class_name.unique())
del class_names[class_names.index('No finding')]
class_names = class_names+['No finding']
classes = dict(zip(list(range(15)),class_names))

In [ ]:
classes

In [ ]:
def prepareDataFrame(train_df= train):
    train_df = train_df.fillna(0)
#     train_df = train_df.head(10)
    
    cols = ['image_id','label']+list(range(4*len(class_names[:-1])))
    return_df = pd.DataFrame(columns=cols)
    
    for image in tqdm(train_df.image_id.unique()):
#         print('image=', image)
        df = train_df.query("image_id==@image")
#         print('df=', df)

        label = np.zeros(15)
        for cls in df.class_id.unique():
#             print('cls=', cls)
            label[int(cls)]=1
#             print('label=', label)
            
        bboxes_df = df.groupby('class_id')[['x_min','y_min','x_max','y_max']].mean().round()
#         print('bboxes_df=', bboxes_df)
        
        bboxes_list = [0 for i in range(60)]
        for ind in list(bboxes_df.index):
            bboxes_list[4*ind:4*ind+4] = list(bboxes_df.loc[ind,:].values)
        return_df.loc[len(return_df),:] = [image]+[label]+bboxes_list[:-4]
        
#         print('===========\n')
        
    return return_df
train_df = prepareDataFrame()

In [ ]:
train_df.head(2)

In [ ]:
train_df.shape

In [ ]:
my_cols = ['image_id',    'label']
train_df = train_df[my_cols]


In [ ]:
from sklearn.model_selection import KFold


In [ ]:
def generateFolds(n_splits = None):
    kf = KFold(n_splits= n_splits)
    for id,(tr_,val_) in enumerate(kf.split(train_df["image_id"],train_df["label"])):
        train_df.loc[val_,'kfold'] = int(id)
    train_df["kfold"].astype(int)

generateFolds(n_splits=5)

In [ ]:
class DataLoader:
    def __init__(self,path = None,train_df=train_df,val_df=None):
        self.path = path
        self.df = train_df
        self.val_df = val_df
        self.train_list = [f'{img}.npy' for img in train_df["image_id"].unique()]
        np.random.shuffle(self.train_list)
        self.test_list = [f'{img}.npy' for img in val_df["image_id"].unique()]
        np.random.shuffle(self.test_list)
    
    def read_image(self):
        for img in self.train_list:
            im_name = img.split('.npy')[0]
            image = np.load(self.path+img)
            temp = self.df[self.df.image_id==im_name]
            c_label,bb = temp.iloc[0,1],temp.iloc[0,2:].values.astype('float')
            yield image,c_label,bb
    
    
    def batch_generator(self,items,batch_size):
        a=[]
        i=0
        for item in items:
            a.append(item)
            i+=1

            if i%batch_size==0:
                yield a
                a=[]
        if len(a) is not 0:
            yield a
            
    def flow(self,batch_size):
        """
        flow from given directory in batches
        ==========================================
        batch_size: size of the batch
        """
        while True:
            for bat in self.batch_generator(self.read_image(),batch_size):
                batch_images = []
                batch_c_labels = []
                batch_bb = []
                for im,im_c_label,im_bb in bat:
                    batch_images.append(im)
                    batch_c_labels.append(im_c_label)
                    batch_bb.append(im_bb)
                batch_images = np.stack(batch_images,axis=0)

#                 batch_labels =  (np.stack(batch_c_labels,axis=0),np.stack(batch_bb,axis=0))
                batch_labels =  np.stack(batch_c_labels,axis=0)
                yield batch_images,batch_labels
    
    def getVal(self):
        images = []
        c_labels = []
        bb_labels = []
        for img in self.test_list:
            im_name = img.split('.npy')[0]
            image = np.load(self.path+img)
            temp = self.val_df[self.val_df.image_id==im_name]
            c_label,bb = temp.iloc[0,1],temp.iloc[0,2:].values.astype('float')
            images.append(image)
            c_labels.append(c_label)
            bb_labels.append(bb)

#         return np.stack(images,axis=0),(np.stack(c_labels,axis=0),np.stack(bb_labels,axis=0))
        return np.stack(images,axis=0),np.stack(c_labels,axis=0)
    

In [ ]:
# class_label = np.zeros((len(X_test),15))
# bb_label = np.zeros((len(X_test),56))

for fold in range(3):
    print(f'\nFold: {fold}\n')
    
#     X_train = train_df[train_df.kfold!=fold].drop('kfold',axis=1)
#     X_val = train_df[train_df.kfold==fold].drop('kfold',axis=1)
    X_train = train_df[train_df.kfold!=fold]
    X_val = train_df[train_df.kfold==fold]
    print('X_train.shape=',  X_train.shape)
    print('X_train.head()=',  X_train.head())
    
    print('-----------\n')
    
    print('X_val.shape=',  X_val.shape)
    print('X_val.head()=',  X_val.head())

    
    print('-----------\n')
    dl = DataLoader('../input/xraynumpy/images/train/',X_train,X_val)
    train_set = dl.flow(batch_size=32)

    X_eval,Y_eval = dl.getVal()
#     print('X_eval[0]=', X_eval[0])
    print('X_eval.shape=', X_eval.shape)

#     print('Y_eval[0]=', Y_eval[0])
    print('Y_eval.shape=', Y_eval.shape)
    
    
    break
    


### now build densenet model

In [ ]:
import tensorflow as tf
# import tensorflow.keras.layers as L
import tensorflow.keras.backend as K


In [ ]:
tf.__version__

In [ ]:
from tensorflow.keras.metrics import Recall, Precision

### get perf metrics per class

In [ ]:
y = Y_eval

In [ ]:
def get_true_pos(y, pred, th=0.5):
    pred_t = (pred > th)
    return np.sum((pred_t == True) & (y == 1))


def get_true_neg(y, pred, th=0.5):
    pred_t = (pred > th)
    return np.sum((pred_t == False) & (y == 0))


def get_false_neg(y, pred, th=0.5):
    pred_t = (pred > th)
    return np.sum((pred_t == False) & (y == 1))


def get_false_pos(y, pred, th=0.5):
    pred_t = (pred > th)
    return np.sum((pred_t == True) & (y == 0))

In [ ]:
def true_positives(y, pred, th=0.5):
    """
    Count true positives.

    Args:
        y (np.array): ground truth, size (n_examples)
        pred (np.array): model output, size (n_examples)
        th (float): cutoff value for positive prediction from model
    Returns:
        TP (int): true positives
    """
    TP = 0
    
    # get thresholded predictions
    thresholded_preds = pred >= th

    # compute TP
    TP = np.sum((y == 1) & (thresholded_preds == 1))
    
    return TP

def true_negatives(y, pred, th=0.5):
    """
    Count true negatives.

    Args:
        y (np.array): ground truth, size (n_examples)
        pred (np.array): model output, size (n_examples)
        th (float): cutoff value for positive prediction from model
    Returns:
        TN (int): true negatives
    """
    TN = 0
    
    # get thresholded predictions
    thresholded_preds = pred >= th

    ### START CODE HERE (REPLACE INSTANCES OF 'None' with your code) ###
    
    # compute TN
    TN = np.sum((y == 0) & (thresholded_preds == 0))

    ### END CODE HERE ###
    
    return TN

def false_positives(y, pred, th=0.5):
    """
    Count false positives.

    Args:
        y (np.array): ground truth, size (n_examples)
        pred (np.array): model output, size (n_examples)
        th (float): cutoff value for positive prediction from model
    Returns:
        FP (int): false positives
    """
    FP = 0
    
    # get thresholded predictions
    thresholded_preds = pred >= th
    
    ### START CODE HERE (REPLACE INSTANCES OF 'None' with your code) ###

    # compute FP
    FP = np.sum((y == 0) & (thresholded_preds == 1))

    ### END CODE HERE ###
    
    return FP

def false_negatives(y, pred, th=0.5):
    """
    Count false positives.

    Args:
        y (np.array): ground truth, size (n_examples)
        pred (np.array): model output, size (n_examples)
        th (float): cutoff value for positive prediction from model
    Returns:
        FN (int): false negatives
    """
    FN = 0
    
    # get thresholded predictions
    thresholded_preds = pred >= th

    ### START CODE HERE (REPLACE INSTANCES OF 'None' with your code) ###
    
    # compute FN
    FN = np.sum((y == 1) & (thresholded_preds == 0))

    ### END CODE HERE ###
    
    return FN

In [ ]:
def get_accuracy(y, pred, th=0.5):
    """
    Compute accuracy of predictions at threshold.

    Args:
        y (np.array): ground truth, size (n_examples)
        pred (np.array): model output, size (n_examples)
        th (float): cutoff value for positive prediction from model
    Returns:
        accuracy (float): accuracy of predictions at threshold
    """
    accuracy = 0.0
    
    ### START CODE HERE (REPLACE INSTANCES OF 'None' with your code) ###
    
    # get TP, FP, TN, FN using our previously defined functions
    TP = true_positives(y, pred, th)
    FP = false_positives(y, pred, th)
    TN = true_negatives(y, pred, th)
    FN = false_negatives(y, pred, th)

    # Compute accuracy using TP, FP, TN, FN
    accuracy = (TP + TN) / (TP + TN + FP + FN) 
    
    ### END CODE HERE ###
    
    return accuracy

In [ ]:
def get_performance_metrics(y, 
                            pred, 
                            class_labels, 
                            tp=get_true_pos,
                            tn=get_true_neg, 
                            fp=get_false_pos,
                            fn=get_false_neg,
                            acc=None, 
                            prevalence=None, 
                            spec=None,
                            sens=None, 
                            ppv=None, 
                            npv=None, 
                            auc=None, 
                            f1=None,
                            thresholds=[]):
    if len(thresholds) != len(class_labels):
        thresholds = [.5] * len(class_labels)

    columns = ["", "TP", "TN", "FP", "FN", "Accuracy", "Prevalence",
               "Sensitivity",
               "Specificity", "PPV", "NPV", "AUC", "F1", "Threshold"]
    df = pd.DataFrame(columns=columns)
    for i in range(len(class_labels)):
        df.loc[i] = [""] + [0] * (len(columns) - 1)
        df.loc[i][0] = class_labels[i]
        df.loc[i][1] = round(tp(y[:, i], pred[:, i]),
                             3) if tp != None else "Not Defined"
        df.loc[i][2] = round(tn(y[:, i], pred[:, i]),
                             3) if tn != None else "Not Defined"
        df.loc[i][3] = round(fp(y[:, i], pred[:, i]),
                             3) if fp != None else "Not Defined"
        df.loc[i][4] = round(fn(y[:, i], pred[:, i]),
                             3) if fn != None else "Not Defined"
        df.loc[i][5] = round(acc(y[:, i], pred[:, i], thresholds[i]),
                             3) if acc != None else "Not Defined"
        df.loc[i][6] = round(prevalence(y[:, i]),
                             3) if prevalence != None else "Not Defined"
        df.loc[i][7] = round(sens(y[:, i], pred[:, i], thresholds[i]),
                             3) if sens != None else "Not Defined"
        df.loc[i][8] = round(spec(y[:, i], pred[:, i], thresholds[i]),
                             3) if spec != None else "Not Defined"
        df.loc[i][9] = round(ppv(y[:, i], pred[:, i], thresholds[i]),
                             3) if ppv != None else "Not Defined"
        df.loc[i][10] = round(npv(y[:, i], pred[:, i], thresholds[i]),
                              3) if npv != None else "Not Defined"
        df.loc[i][11] = round(auc(y[:, i], pred[:, i]),
                              3) if auc != None else "Not Defined"
        df.loc[i][12] = round(f1(y[:, i], pred[:, i] > thresholds[i]),
                              3) if f1 != None else "Not Defined"
        df.loc[i][13] = round(thresholds[i], 3)

    df = df.set_index("")
    return df

In [ ]:
# classes

In [ ]:
class_labels = list(classes.values())

In [ ]:
class_labels

In [ ]:
from sklearn.metrics import roc_auc_score, f1_score

### need to improve this model

In [ ]:
# Import Densenet from Keras
from keras.applications.densenet import DenseNet121
from keras.layers import Dense, GlobalAveragePooling2D
from keras.models import Model
from keras import backend as K

In [ ]:
def build_densenet_model():
    # get input layer
    img_input = tf.keras.layers.Input(shape=(256, 256,1))

    # change shape for compatibility
    img_conc = tf.keras.layers.Concatenate()([img_input, img_input, img_input])     
    
    # load base model - transfer learned
    base_model = DenseNet121(weights='../input/densenet-weights-nih-coursera-ai4m/densenet.hdf5', include_top=False, input_tensor=img_conc)
    
    # see last layer - customise it
    x = base_model.output
    
    # Add a global spatial average pooling layer
    x_pool = GlobalAveragePooling2D()(x)

    # Add a logistic layer the same size as the number of classes you're trying to predict
    n_classes = 15

    predictions = Dense(n_classes, activation="sigmoid")(x_pool)
    print(f"Predictions have {n_classes} units, one for each class")

    # Create an updated model
    # model = Model(inputs=in1, outputs=predictions)
    model = Model(inputs=img_input, outputs=predictions)
    
    return model


In [ ]:
model2 = build_densenet_model()

In [ ]:
# Compile the model
model2.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy', tf.keras.metrics.AUC()]
             )

In [ ]:
    chckpt = tf.keras.callbacks.ModelCheckpoint(f'./model2.1_march28.hdf5',
                                                monitor='val_loss',
                                                mode='min',
                                                save_best_only=True)


In [ ]:
# train_set.shape

In [ ]:
class_labels

In [ ]:
for k, v in dict(train_reliable2.class_name.value_counts()).items():
    if v < 1000:
        print('minority class=', k, v)

In [ ]:
weights = {0:1, 
           1:2, 
           2:2,
           3:1,
           4:2,
           5:2,
           6:1,
           7:1,
           8:1,
           9:1,
           10:1,
           11:1,
           12:2,
           13:1,
           14:1
          }

In [ ]:
    print('-----------\n')
    model2.fit(train_set,
               epochs=5,
               steps_per_epoch=int(7500/32),
               validation_data = (X_eval,Y_eval),
               callbacks = [chckpt],
               class_weight=weights
              )


In [ ]:
# ! ls ../input/chest-x-ray-abnormalities-densenet-pipeline
! ls

In [ ]:
# model2.load_weights('../input/chest-x-ray-abnormalities-densenet-pipeline/model2.hdf5')
model2.load_weights('./model2.1_march28.hdf5')


In [ ]:
# model2

In [ ]:
model2.evaluate(X_eval, y)

In [ ]:
y_pred2 = model2.predict(X_eval)


In [ ]:
get_performance_metrics(y, 
                        y_pred2, 
                        class_labels, 
                        acc=get_accuracy, 
                        auc=roc_auc_score,
                        f1=f1_score)

### new model - change metrics.

In [ ]:
# model3 = build_densenet_model()

In [ ]:
# # Compile the model
# model3.compile(optimizer='adam',
#                loss='binary_crossentropy',
#                metrics=[tf.keras.metrics.AUC(), 'accuracy']
#               )

In [ ]:
#     chckpt = tf.keras.callbacks.ModelCheckpoint(f'./model3.hdf5',
#                                                 monitor='val_loss',
#                                                 mode='min',
#                                                 save_best_only=True)


In [ ]:
#     print('-----------\n')
#     model3.fit(train_set,
#               epochs=5,
#               steps_per_epoch=int(7500/32),
#               validation_data = (X_eval,Y_eval),
#               callbacks = [chckpt]
#              )


In [ ]:
# model3.evaluate(X_eval, y)

In [ ]:
# model3.load_weights('./model3.hdf5')

In [ ]:
# model3.evaluate(X_eval, y)

In [ ]:
# y_pred3 = model3.predict(X_eval)


In [ ]:
# get_performance_metrics(y, 
#                         y_pred3, 
#                         class_labels, 
#                         acc=get_accuracy, 
#                         auc=roc_auc_score,
#                         f1=f1_score)

### explore, eval predictions

In [ ]:
train_df.head(20)

In [ ]:
# filename = '03e6ecfa6f6fb33dfeac6ca4f9b459c9'
# filename = '9a5094b2563a1ef3ff50dc5c7ff71345'
# filename = '47ed17dcb2cbeec15182ed335a8b5a9e'
# filename = 'c394eadea89e5795c8037280492d116d'
filename = '22b8e616a61bbc4caaed0cf23b7159df'
# filename = '80caa435b6ab5edaff4a0a758ffaec6e'







In [ ]:
docs0 = train[train.image_id == filename]

In [ ]:
docs0

In [ ]:
# train.shape

In [ ]:
# train_df.shape

In [ ]:
docs = train_df[train_df.image_id == filename]

In [ ]:
docs

In [ ]:
docs.iloc[0]

In [ ]:
# docs["image_id"].unique()

In [ ]:
# for img in docs["image_id"].unique():
#     print(img)

In [ ]:
train_list = [f'{img}.npy' for img in docs["image_id"].unique()]


In [ ]:
# train_list

In [ ]:
path = '../input/xraynumpy/images/train/'

In [ ]:
for img in docs["image_id"].unique():
    print(img)
    image = np.load(path+img + '.npy')


In [ ]:
# image.shape

In [ ]:
# image

In [ ]:
model = model2

In [ ]:
images = [image]

X = np.stack(images,axis=0)



In [ ]:
op = model.predict(X)


In [ ]:
# classes

In [ ]:
# list(op)

In [ ]:
op = op[0]

In [ ]:
op

In [ ]:
for ix, y_pred in enumerate(list(op)):
    if y_pred > 0.5:
        print(ix, y_pred, classes[ix])

In [ ]:
for ix, y in enumerate(list(docs.iloc[0].label)):
    if y > 0:
        print(ix, y, classes[ix])

In [ ]:
docs0.class_name.value_counts()